In [134]:
import numpy as np
import pandas as pd

import requests

from selenium import webdriver
from bs4 import BeautifulSoup

from time import sleep

from datetime import datetime

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

import re

In [7]:
df_perfil_especialidad = pd.read_csv("StackMinimo - StackTec.csv", skiprows = 2).drop("Unnamed: 0", axis = 1)

df_perfil_especialidad

,Codigo,N1 Perfil,N2 Especialidad,N3 Tecnología predominante
0,DESBACK,Desarrollador,Backend,.Net
1,NaN,NaN,NaN,Java
2,NaN,NaN,NaN,NodeJs
3,NaN,NaN,NaN,Phyton
4,NaN,NaN,NaN,QuickBase
5,NaN,NaN,NaN,"Lanbda, Azure Functions, GCP Functions"
6,DESBD,Desarrollador,Base de datos,Oracle Forms
7,NaN,NaN,NaN,SQL
8,NaN,NaN,NaN,SAP
9,DESAPI,Desarrollador,APIs,All


In [40]:
list_especialidad = df_perfil_especialidad["N2 Especialidad"].dropna().to_list()

In [74]:
list_especialidad = ["Backend",
                     "Base de datos",
                     "Bases de datos",
                     "APIs",
                     "Mobile",
                     "Frontend Movil",
                     "Frontend Web",
                     "Full Stack",
                     "Integración",
                     "Liferay",
                     "Power BI",
                     "Software Release Engineer",
                     "SRE",
                     "Diseñador Gráfico",
                     "Graphic Designer",
                     "Infraestructura cloud",
                     "Estructuras cloud",
                     "Sistemas Operativos",
                     "Seguridad",
                     "Mantenimiento y Soporte",
                     "Administrador de Base de datos",
                     "Servidores y aplicaciones",
                     "Tester funcional",
                     "Tester automatizada",
                     "SCRUM Master",
                     "Product Owner",
                     "Customer Success",
                     "Gerente de Proyecto",
                     "Analisis de Datos",
                     "Analista de Datos"]

In [ ]:
{"Base de datos" : "Base de datos",
 "Bases de datos" : "Base de datos"}

In [75]:
dict_perfiles = {v : k for k, v in df_perfil_especialidad[["N1 Perfil ", "N2 Especialidad"]].dropna().values}

In [77]:
dict_perfiles = {"Backend"                   : "Desarrollador",
                 "Base de datos"             : "Desarrollador",
                 "APIs"                      : "Desarrollador",
                 "Mobile"                    : "Desarrollador",
                 "Frontend Movil"            : "Desarrollador",
                 "Frontend Web"              : "Desarrollador",
                 "Full Stack"                : "Desarrollador",
                 "Integración"               : "Desarrollador",
                 "Liferay"                   : "Desarrollador",
                 "Power BI"                  : "Desarrollador",
                 "Software Release Engineer" : "Devops",
                 "Diseñador Gráfico"         : "Diseñador",
                 "Infraestructura cloud"     : "Infraestructura",
                 "Sistemas Operativos"       : "Infraestructura",
                 "Seguridad"                 : "Infraestructura",
                 "Mantenimiento y Soporte"   : "Infraestructura",
                 "Tester funcional"          : "Quality Assurance",
                 "Tester automatizada"       : "Quality Assurance",
                 "SCRUM Master"              : "Gestión Operativa",
                 "Product Owner"             : "Gestión Operativa",
                 "Customer Success"          : "Gestión Operativa",
                 "Gerente de Proyecto"       : "Gestión Operativa",
                 "Analisis de Datos"         : "Especialista"}

In [2]:
hab1 = AirtableHAB(table_name = "LATAM 001")
hab2 = AirtableHAB(table_name = "LATAM 002")
hab3 = AirtableHAB(table_name = "LATAM 003")

In [8]:
%%time

df1, df2, df3 = hab1.airtable_get(), hab2.airtable_get(), hab3.airtable_get()

print(df1.shape, df2.shape, df3.shape)

(50000, 42) (23689, 22) (50000, 14)
CPU times: total: 38.8 s
Wall time: 18min 56s


In [13]:
print(df1.drop_duplicates("description").shape)
print(df2.drop_duplicates("description").shape)
print(df3.drop_duplicates("description").shape)

(27262, 42)
(13577, 22)
(24239, 14)


In [20]:
27262 + 13577 + 24239

65078

In [214]:
def get_especialidad(string, list_especialidad):
    
    especialidades = list({especialidad for especialidad in list_especialidad if especialidad.lower() in string.lower()})
    
    return especialidades if especialidades else []

In [215]:
def get_perfil(lista, dict_perfiles):
    
    perfiles = list({v for k, v in dict_perfiles.items() if k in lista})
    
    return perfiles if perfiles else []

In [216]:
%%time

df1["Especialidad"] = df1["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)
df2["Especialidad"] = df2["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)
df3["Especialidad"] = df3["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)

CPU times: total: 53.5 s
Wall time: 1min 50s


In [217]:
%%time

df1["Perfil"] = df1["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)
df2["Perfil"] = df2["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)
df3["Perfil"] = df3["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)

CPU times: total: 5.62 s
Wall time: 13.5 s


In [194]:
prueba = df1[["airtable_id", "Especialidad", "Perfil"]]

prueba

,airtable_id,Especialidad,Perfil
0,rec007ki0rty4r45D,[],[]
1,rec008d5T9DjWTVBw,[Mobile],[]
2,rec009ePhEZgwdUZF,"[Seguridad, Backend, Mobile, Full Stack, APIs]",[Desarrollador]
3,rec00F8BVyguzTbVL,[],[]
4,rec00IDNdY6vDxqDb,"[Product Owner, Mobile]","[Desarrollador, Gestión Operativa]"
...,...,...,...
49995,reczzkEh5UM4jSMYJ,"[Power BI, Seguridad]",[Desarrollador]
49996,reczzmUDDTai3yyxs,[],[]
49997,reczzn8dxb0KvZCcb,[],[]
49998,reczznz9goqSvw6yL,[],[]


In [218]:
prueba["Especialidad"] = prueba["Especialidad"].apply(lambda x : x if len(x) > 0 else [None])
prueba["Perfil"] = prueba["Perfil"].apply(lambda x : x if len(x) > 0 else [None])

C:\Users\danie\AppData\Local\Temp\ipykernel_528\846656297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Especialidad"] = prueba["Especialidad"].apply(lambda x : x if len(x) > 0 else [None])
C:\Users\danie\AppData\Local\Temp\ipykernel_528\846656297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Perfil"] = prueba["Perfil"].apply(lambda x : x if len(x) > 0 else [None])


In [ ]:
# %%time

# for i in range(df1.shape[0]):
    
#     records = {"records" : []}
    
#     row = df1.iloc[i].to_dict()
    
#     id_ = row["airtable_id"]
    
#     row.pop("airtable_id")
    
#     records["records"].append({"id"     : id_,
#                                "fields" : {k : v for k, v in row.items()}})
    
#     records["typecast"] = True

#     # LATAM 001
#     response = requests.patch(url = hab1.endpoint, json = records, headers = hab1.headers)

#     print(f"response: {response.status_code}")
    

In [207]:
df__1 = df1[["airtable_id", "Especialidad", "Perfil"]]
df__1.columns = ["airtable_id", "Especialidad_Multiple", "Perfil_Multiple"]

for i in range(0, df__1.shape[0], 10):
    
    data = df__1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : data.iloc[x, 1:].to_dict()} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}
    
    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
#     print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [212]:
%%time
df__2 = df2[["airtable_id", "Especialidad", "Perfil"]]
df__2.columns = ["airtable_id", "Especialidad_Multiple", "Perfil_Multiple"]

for i in range(0, df__2.shape[0], 10):
    
    data = df__2.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : data.iloc[x, 1:].to_dict()} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}
    
    response = requests.patch(url = hab2.endpoint, json = data, headers = hab2.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200


IndexError: index 9 is out of bounds for axis 0 with size 9

In [219]:
%%time
df__3 = df3[["airtable_id", "Especialidad", "Perfil"]]
df__3.columns = ["airtable_id", "Especialidad_Multiple", "Perfil_Multiple"]

for i in range(0, df__3.shape[0], 10):
    
    data = df__3.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : data.iloc[x, 1:].to_dict()} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}
    
    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [ ]:
################################################################################################################################

In [308]:
%%time
df_n1 = pd.read_csv("extract_n_1.csv")
df_n2 = pd.read_csv("extract_n_2.csv")
df_n3 = pd.read_csv("extract_n_3.csv")
df_n4 = pd.read_csv("extract_n_4.csv")
df_n5 = pd.read_csv("extract_n_5.csv")
df_n6 = pd.read_csv("extract_n_6.csv")
df_n7 = pd.read_csv("extract_n_7.csv")
df_n8 = pd.read_csv("extract_n_8.csv")
df_n9 = pd.read_csv("extract_n_9.csv")
df_n10 = pd.read_csv("extract_n_10.csv")
df_n11 = pd.read_csv("extract_n_11.csv")
df_n12 = pd.read_csv("extract_n_12.csv")
df_n13 = pd.read_csv("extract_n_13.csv")
df_n14 = pd.read_csv("extract_n_LAST_CHUNK.csv")

CPU times: total: 2.7 s
Wall time: 4.12 s


In [256]:
df_n1 = df_n1[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df__1 = df__1.merge(df_n1)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]
df__1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df__1["chat_gpt"] = "Chat-Gpt"

,airtable_id,Especialidad,Perfil,job_id
0,rec007ki0rty4r45D,[],[],eyJqb2JfdGl0bGUiOiJQcm9ncmFtYWRvciBVbml0eSIsIm...
1,rec008d5T9DjWTVBw,[Mobile],[],eyJqb2JfdGl0bGUiOiJUcmFiYWpvIERlc2RlIENhc2EgRG...
2,rec009ePhEZgwdUZF,"[Seguridad, Backend, Mobile, Full Stack, APIs]",[Desarrollador],eyJqb2JfdGl0bGUiOiJQcm9ncmFtYWRvciBXZWIiLCJodG...
3,rec00F8BVyguzTbVL,[],[],eyJqb2JfdGl0bGUiOiJBbmFsaXN0YSBkZSBNYXJrZXRpbm...
4,rec00IDNdY6vDxqDb,"[Product Owner, Mobile]","[Desarrollador, Gestión Operativa]",eyJqb2JfdGl0bGUiOiJQcm9ncmFtYWRvciBQSFAgLSBMQV...
...,...,...,...,...
49995,reczzkEh5UM4jSMYJ,"[Power BI, Seguridad]",[Desarrollador],eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBERSBQUk9HUkFNQU...
49996,reczzmUDDTai3yyxs,[],[],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIFB5dGhvbi...
49997,reczzn8dxb0KvZCcb,[],[],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIEZ1bGxzdG...
49998,reczznz9goqSvw6yL,[],[],eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgUHJpb3JpdHkiLC...


In [260]:
for i in range(0, df__1.shape[0], 10):
    
    data = df__1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 1 is out of bounds for axis 0 with size 1

In [264]:
df_n2 = df_n2[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df__1 = df__1.merge(df_n2)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]
df__1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df__1["chat_gpt"] = "Chat-Gpt"

df__2

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt


In [270]:
df_n2 = df_n2[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n2)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00eeEGTyJEwrDQ,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec00pWhNAnfH3O52,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec00xg8Lakz4Cwq6,2500000.0,3000000.0,COP,month,Chat-Gpt
3,rec01HCXsiyOPRkQY,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec01NIOT0Ay9kVUp,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4995,reczxhC4HRxR4atlF,0.0,0.0,NaN,NaN,Chat-Gpt
4996,reczxw1Ck5HFdvO5n,0.0,0.0,NaN,NaN,Chat-Gpt
4997,reczy2Js6xcuykYp6,0.0,0.0,NaN,NaN,Chat-Gpt
4998,reczyzzt1QrbZ85xl,0.0,0.0,NaN,NaN,Chat-Gpt


In [271]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [272]:
df_n3 = df_n3[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n3)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec014g7ppqaolm3O,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec02C44lNdpJbWAl,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec02kdLqLX29Lan0,0.0,0.0,NaN,NaN,Chat-Gpt
3,rec02rZ4q1due6AR0,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec040WbfeQKr6w1S,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4995,reczy3YdR8DHlwfLX,0.0,0.0,NaN,NaN,Chat-Gpt
4996,reczzd05d1HePrhpI,0.0,0.0,NaN,NaN,Chat-Gpt
4997,reczzmUDDTai3yyxs,0.0,0.0,NaN,NaN,Chat-Gpt
4998,reczzn8dxb0KvZCcb,0.0,0.0,NaN,NaN,Chat-Gpt


In [273]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [275]:
df_n4 = df_n4[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n4)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00uEdKuoORinzh,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec01d5KZqX8bieBb,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec01iNFQK1UR2p5m,0.0,0.0,NaN,NaN,Chat-Gpt
3,rec01wOaLTnEslp6w,650000.0,650000.0,CLP,month,Chat-Gpt
4,rec05kAHQ9o77xrD6,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4995,reczs9QUMTCpaXrHI,0.0,0.0,NaN,NaN,Chat-Gpt
4996,reczsj21yHyz8klaY,3500000.0,3500000.0,COP,month,Chat-Gpt
4997,reczuIGaBGDZ1JE3X,0.0,0.0,NaN,NaN,Chat-Gpt
4998,reczxGJ4ukm5Qq6kA,0.0,0.0,NaN,NaN,Chat-Gpt


In [276]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [280]:
df_n5 = df_n5[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df3[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n5)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00nm400BSI8rtO,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec021kWyjF0HZMKl,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec02osINQRbedmnc,0.0,0.0,NaN,NaN,Chat-Gpt
3,rec03yV6LpIfYwz4l,6000000.0,8000000.0,COP,month,Chat-Gpt
4,rec05ILaRyaPQA12a,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4986,reczw1mPVA0aL2qEO,0.0,0.0,NaN,NaN,Chat-Gpt
4987,reczwWUMjtf0zM6F6,0.0,0.0,NaN,NaN,Chat-Gpt
4988,reczwjZwyiV4mTY9S,0.0,0.0,NaN,NaN,Chat-Gpt
4989,reczwy9P4rqcWlgO7,600.0,600.0,USD,month,Chat-Gpt


In [286]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 1 is out of bounds for axis 0 with size 1

In [285]:
response.json()

{'error': {'type': 'UNKNOWN_FIELD_NAME',
  'message': 'Unknown field name: "chat_gpt"'}}

In [287]:
df_n6 = df_n6[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df3[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n6)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00J9cnH3dr5gdG,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec05ucjrdCNy0jP7,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec06WUt7rH85hbtS,0.0,0.0,NaN,NaN,Chat-Gpt
3,rec06XNViqntMMck4,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec06hkuyJuxC4zjz,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4995,reczv3QAcngukI4Ih,0.0,0.0,NaN,NaN,Chat-Gpt
4996,reczvRXx7VQqkGWmX,0.0,0.0,NaN,NaN,Chat-Gpt
4997,reczweQV7kqissgqp,62000.0,66000.0,USD,month,Chat-Gpt
4998,reczwuINLmvvYDZgV,0.0,0.0,NaN,NaN,Chat-Gpt


In [288]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [290]:
df_n7 = df_n7[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df3[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n7)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00brgJ5fYFOZmw,0.00,0.00,NaN,NaN,Chat-Gpt
1,rec00laEnL5fbavrO,150.00,300.00,USD,week,Chat-Gpt
2,rec00llD2yETHV7Rn,6300.00,6300.00,USD,month,Chat-Gpt
3,rec00yJI3hfP1JKID,0.00,0.00,NaN,NaN,Chat-Gpt
4,rec01WtQplnAiNrju,0.00,0.00,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4994,reczy075vvcD2o7NF,13360.67,13360.67,MXN,month,Chat-Gpt
4995,reczy4P0I9rAwW5Gn,0.00,0.00,NaN,NaN,Chat-Gpt
4996,reczyLu8wRc2pJMAi,0.00,0.00,NaN,NaN,Chat-Gpt
4997,reczyUpejFj587ZRS,0.00,0.00,NaN,NaN,Chat-Gpt


In [291]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 9 is out of bounds for axis 0 with size 9

In [292]:
response.json()

{'records': [{'id': 'reczoILA1Q9W4Fiye',
   'createdTime': '2023-07-31T04:39:02.000Z',
   'fields': {'job_id': 'eyJqb2JfdGl0bGUiOiJEaWdpdGFsIFNvbHV0aW9ucyB8RGV2b3BzIEVuZ2luZWVyIiwiaHRpZG9jaWQiOiJ5bkZ5TE1DM3pMWUFBQUFBQUFBQUFBPT0iLCJ1dWxlIjoidytDQUlRSUNJSlIzVmhkR1Z0WVd4aCIsImZjIjoiRXFJQ0N1SUJRVTFyTW14c1Mwb3dOV0kyWkUxTFdGRlRTRFowVVY4MlQwRnFZa1JUTTNOVlkxOVJSV1pVZVROdFYwTkhXazFLWVZST09EUmhXVUpNYmw5SlJtTlRORk5DWTBscFVrNXNVV1UwUkZWTFdESnRjVTVUYUcwMU9FNUhlVWQzTjFacFNIWnBWakJpTW5aNGJFMUdaelZEVkdVNWJUbDVVRkY0UVMxekxVMXlUWFkwWm5FdFZ6QlhjV1JYYlRCRVkxVlJlRWhHVmsxblRYRjVhME5YV2xoUWNVODNRVWxCUlVKb2IxQmZUa3RKUnpkS01rWm5WbVUzY3pkU1V6QXlVMFZDT0VadU1uRlRTekZPY2w5MlVtWlpibmhSUW5oSlJHMWFRbWR6UkdSUFZYUTNaeElYU25GNlFscE1abWxOY0V0dWNYUnpVRFU1UjB3dFFXY2FJa0ZNUlZNNWRVOUpWVmxxVG1ScVQxWmtOME5KVDJaU2VISmZjSE5DYW14YVYyYyIsImZjdiI6IjMiLCJmY19pZCI6ImZjXzE0IiwiYXBwbHlfbGluayI6eyJ0aXRsZSI6IkFwcGx5IG9uIExpbmtlZEluIEd1YXRlbWFsYSIsImxpbmsiOiJodHRwczovL2d0LmxpbmtlZGluLmNvbS9qb2JzL3ZpZXcvZGlnaXRhbC1zb2x1dGlvbnMtZGV2b3BzLWVu

In [293]:
df_n8 = df_n8[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df3[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n8)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec000U1PUGICWE45,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec00Hzg68SZbumzk,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec00frd0A5cN9lum,8000.0,8000.0,MXN,month,Chat-Gpt
3,rec02afOWrSK40ZFu,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec02kR1kh87D8oTT,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4995,reczyDBVIhxOtR4zr,10000.0,10000.0,MXN,month,Chat-Gpt
4996,reczyIFmBz0VUfARZ,0.0,0.0,NaN,NaN,Chat-Gpt
4997,reczyyQuH11bZePJN,62000.0,62000.0,MXN,month,Chat-Gpt
4998,reczzb3KFyjIcZMXd,0.0,0.0,NaN,NaN,Chat-Gpt


In [294]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [296]:
df_n9 = df_n9[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df3[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n9)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00J9cnH3dr5gdG,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec00tjW8LTUxzOmd,25000.0,30000.0,USD,month,Chat-Gpt
2,rec01axgkbut3blQY,0.0,0.0,NaN,NaN,Chat-Gpt
3,rec021kWyjF0HZMKl,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec02oilZrA4eJZVB,7500.0,8500.0,USD,month,Chat-Gpt
...,...,...,...,...,...,...
4996,reczvv7MO877WNHRk,20000.0,26000.0,USD,month,Chat-Gpt
4997,reczwWUMjtf0zM6F6,0.0,0.0,NaN,NaN,Chat-Gpt
4998,reczyB279T6SOtvni,0.0,0.0,NaN,NaN,Chat-Gpt
4999,reczyUpejFj587ZRS,0.0,0.0,NaN,NaN,Chat-Gpt


In [297]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab3.endpoint, json = data, headers = hab3.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 1 is out of bounds for axis 0 with size 1

In [309]:
df_n10 = df_n10[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df2[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n10)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00D3ZkHNh9B9hB,0.0,0.0,NaN,NaN,Chat-Gpt
1,rec01KZudXLuFp9Fy,0.0,0.0,NaN,NaN,Chat-Gpt
2,rec04bnp8qng5ME8z,480000.0,480000.0,CLP,month,Chat-Gpt
3,rec053sZz7XQ9nrKW,0.0,0.0,NaN,NaN,Chat-Gpt
4,rec069BJiTPoAWK0l,0.0,0.0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
4364,reczxhfQyBbXvOXTl,0.0,0.0,NaN,NaN,Chat-Gpt
4365,reczxzbMfvJ9KIaiU,70000.0,80000.0,MXN,month,Chat-Gpt
4366,reczy4eyj0qXiNoKN,0.0,0.0,NaN,NaN,Chat-Gpt
4367,reczzR6YSAdfjOBmg,0.0,0.0,NaN,NaN,Chat-Gpt


In [310]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab2.endpoint, json = data, headers = hab2.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 9 is out of bounds for axis 0 with size 9

In [302]:
df_n11 = df_n11[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df2[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n11)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00gEQFqRnfvJQW,0.0,0.0,,,Chat-Gpt
1,rec01VUMWvl4lKvrR,0.0,0.0,,,Chat-Gpt
2,rec01x5NQ0NCpe7E8,0.0,0.0,,,Chat-Gpt
3,rec02SCMUCXtmLVRa,0.0,0.0,,,Chat-Gpt
4,rec02XPa11nbAizvN,0.0,0.0,,,Chat-Gpt
...,...,...,...,...,...,...
4995,reczyh4YYuvPt5JwO,50000.0,55000.0,USD,month,Chat-Gpt
4996,reczyphI9DBK5AD2L,0.0,0.0,,,Chat-Gpt
4997,reczz9ZJnAu50xV2g,0.0,0.0,,,Chat-Gpt
4998,reczzFQD3p4n5s83W,0.0,0.0,,,Chat-Gpt


In [303]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab2.endpoint, json = data, headers = hab2.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [304]:
df_n12 = df_n12[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df2[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n12)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec00Hs3DVElXSzOj,0,0,NaN,NaN,Chat-Gpt
1,rec04ltrN7m92pE2D,0,0,NaN,NaN,Chat-Gpt
2,rec079nD3KnwG17k4,0,0,NaN,NaN,Chat-Gpt
3,rec07biQKB0wjQZTm,0,0,NaN,NaN,Chat-Gpt
4,rec07uOjqcx9TeG8a,0,0,NaN,NaN,Chat-Gpt
...,...,...,...,...,...,...
2593,reczvg1Yrcen6H2J3,1500,1500,PEN,month,Chat-Gpt
2594,reczxQsWziZVdUCN0,0,0,NaN,NaN,Chat-Gpt
2595,reczyV53kbCD84hcS,0,0,NaN,NaN,Chat-Gpt
2596,reczz77ZaHEDoYeii,0,0,NaN,NaN,Chat-Gpt


In [305]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab2.endpoint, json = data, headers = hab2.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 8 is out of bounds for axis 0 with size 8

In [311]:
df_n13 = df_n13[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n13)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec007ki0rty4r45D,0.0,0.0,,,Chat-Gpt
1,rec009ePhEZgwdUZF,0.0,0.0,,,Chat-Gpt
2,rec00fd6PttpXDDCr,0.0,0.0,,,Chat-Gpt
3,rec02Prs1fEUOP1Ad,1650000.0,1650000.0,COP,month,Chat-Gpt
4,rec02rd6yTmTNV2vq,8000.0,12000.0,MXN,month,Chat-Gpt
...,...,...,...,...,...,...
4333,reczveGTN3lHLzGbN,0.0,0.0,,,Chat-Gpt
4334,reczwAlg033nKmbTA,700000.0,700000.0,CLP,month,Chat-Gpt
4335,reczxHvkbPjGb7CHS,0.0,0.0,,,Chat-Gpt
4336,reczylzQKcqoLoITF,0.0,0.0,,,Chat-Gpt


In [313]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(10)]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 8 is out of bounds for axis 0 with size 8

In [314]:
df_n14 = df_n14[["salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0", "job_id"]]
df__1 = df1[["airtable_id", "Especialidad", "Perfil", "job_id"]]
df___1 = df__1.merge(df_n14)[["airtable_id", "salary_min_gpt_0.8.0", "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

df___1.columns = ["airtable_id", "salary_min", "salary_max", "currency", "time_lapse"]
df___1["chat_gpt"] = "Chat-Gpt"

df___1

,airtable_id,salary_min,salary_max,currency,time_lapse,chat_gpt
0,rec008d5T9DjWTVBw,0.0,0.0,,,Chat-Gpt
1,rec01oeKYMAATbNiZ,0.0,0.0,,,Chat-Gpt
2,rec029Nw6mLS3jJgY,0.0,0.0,,,Chat-Gpt
3,rec02MX4nbABlD97r,3500.0,3500.0,GTQ,month,Chat-Gpt
4,rec03FkoHbJzKFq0L,0.0,0.0,,,Chat-Gpt
...,...,...,...,...,...,...
1561,reczofXHC9WIUmjHa,0.0,0.0,,,Chat-Gpt
1562,reczrIIks8gXWdg9w,0.0,0.0,,,Chat-Gpt
1563,reczryYzoT7jdu4b7,0.0,0.0,,,Chat-Gpt
1564,recztjNHAPhNJ1Oih,0.0,0.0,,,Chat-Gpt


In [315]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(len(data))]
    
    data = {"records" : data, "typecast" : True}

    response = requests.patch(url = hab1.endpoint, json = data, headers = hab1.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

IndexError: index 6 is out of bounds for axis 0 with size 6

In [3]:
import pandas as pd
df_sap = pd.read_csv("Competencias Técnicas-Todas las competencias técnicas.csv")

In [6]:
df_sap

,Competencia,Roles,Cursos,Clasificación,Especificación,Tipo de desarrollo,Asociado a...
0,.NET,NaN,NaN,NaN,NaN,NaN,NaN
1,.NETCore,NaN,NaN,Framework,NaN,Back-end,C#
2,Abstract Factory,NaN,NaN,Patrones de diseño y Arquitectura del software,Patrón de creación,Back-end,NaN
3,Aceptación,QA Tester,NaN,Testing,Tipo de testing,NaN,NaN
4,Acknowledgement,NaN,NaN,Patrón de resiliencia,NaN,NaN,NaN
...,...,...,...,...,...,...,...
298,XML,NaN,NaN,API,Tipo de API,NaN,NaN
299,YAGNI,NaN,NaN,Buena práctica,Clean Code,NaN,NaN
300,Zabbix,NaN,NaN,Monitorización,NaN,NaN,NaN
301,Zebra,NaN,NaN,Framework,NaN,Front-end,"HTML,CSS"


In [13]:
df_sap = pd.concat([df_sap, pd.DataFrame(data = [["SAP"], ["UX"]], columns = ["Competencia"])], ignore_index = True)

df_sap.to_csv("Competencias Técnicas-Todas las competencias técnicas.csv", index = False)